# Adding a new code to the GUI

In [1]:
import json
import os

from panqec.codes import StabilizerCode
from panqec.gui import GUI

## Create code class

Let's say you want to implement your own version of the 3D toric code. You should start by creating the Python class of the code.
It should inherit from the base class `StabilizerCode` and have the following structure:

In [2]:
class MyToric3DCode(StabilizerCode):
    dimension = 3

    @property
    def label(self):
        return 'My Toric {}x{}x{}'.format(*self.size)

    def get_qubit_coordinates(self):
        coordinates = []
        Lx, Ly, Lz = self.size

        # Qubits along e_x
        for x in range(1, 2*Lx, 2):
            for y in range(0, 2*Ly, 2):
                for z in range(0, 2*Lz, 2):
                    coordinates.append((x, y, z))

        # Qubits along e_y
        for x in range(0, 2*Lx, 2):
            for y in range(1, 2*Ly, 2):
                for z in range(0, 2*Lz, 2):
                    coordinates.append((x, y, z))

        # Qubits along e_z
        for x in range(0, 2*Lx, 2):
            for y in range(0, 2*Ly, 2):
                for z in range(1, 2*Lz, 2):
                    coordinates.append((x, y, z))

        return coordinates

    def get_stabilizer_coordinates(self):
        coordinates = []
        Lx, Ly, Lz = self.size

        # Vertices
        for x in range(0, 2*Lx, 2):
            for y in range(0, 2*Ly, 2):
                for z in range(0, 2*Lz, 2):
                    coordinates.append((x, y, z))

        # Face in xy plane
        for x in range(1, 2*Lx, 2):
            for y in range(1, 2*Ly, 2):
                for z in range(0, 2*Lz, 2):
                    coordinates.append((x, y, z))

        # Face in yz plane
        for x in range(0, 2*Lx, 2):
            for y in range(1, 2*Ly, 2):
                for z in range(1, 2*Lz, 2):
                    coordinates.append((x, y, z))

        # Face in xz plane
        for x in range(1, 2*Lx, 2):
            for y in range(0, 2*Ly, 2):
                for z in range(1, 2*Lz, 2):
                    coordinates.append((x, y, z))

        return coordinates

    def stabilizer_type(self, location):
        if not self.is_stabilizer(location):
            raise ValueError(f"Invalid coordinate {location} for a stabilizer")

        x, y, z = location
        if x % 2 == 0 and y % 2 == 0:
            return 'vertex'
        else:
            return 'face'

    def get_stabilizer(self, location, deformed_axis=None):
        if not self.is_stabilizer(location):
            raise ValueError(f"Invalid coordinate {location} for a stabilizer")

        if self.stabilizer_type(location) == 'vertex':
            pauli = 'Z'
        else:
            pauli = 'X'

        deformed_pauli = {'X': 'Z', 'Z': 'X'}[pauli]

        x, y, z = location

        if self.stabilizer_type(location) == 'vertex':
            delta = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]
        else:
            # Face in xy-plane.
            if z % 2 == 0:
                delta = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0)]
            # Face in yz-plane.
            elif (x % 2 == 0):
                delta = [(0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]
            # Face in zx-plane.
            elif (y % 2 == 0):
                delta = [(-1, 0, 0), (1, 0, 0), (0, 0, -1), (0, 0, 1)]

        operator = dict()
        for d in delta:
            Lx, Ly, Lz = self.size
            qubit_location = ((x + d[0]) % (2*Lx), (y + d[1]) % (2*Ly), (z + d[2]) % (2*Lz))

            if self.is_qubit(qubit_location):
                is_deformed = (self.qubit_axis(qubit_location) == deformed_axis)
                operator[qubit_location] = deformed_pauli if is_deformed else pauli

        return operator

    def qubit_axis(self, location):
        x, y, z = location

        if (z % 2 == 0) and (x % 2 == 1) and (y % 2 == 0):
            axis = 'x'
        elif (z % 2 == 0) and (x % 2 == 0) and (y % 2 == 1):
            axis = 'y'
        elif (z % 2 == 1) and (x % 2 == 0) and (y % 2 == 0):
            axis = 'z'
        else:
            raise ValueError(f'Location {location} does not correspond to a qubit')

        return axis

    def get_logicals_x(self):
        """The 3 logical X operators."""

        Lx, Ly, Lz = self.size
        logicals = []

        # X operators along x edges in x direction.
        operator = dict()
        for x in range(1, 2*Lx, 2):
            operator[(x, 0, 0)] = 'X'
        logicals.append(operator)

        # X operators along y edges in y direction.
        operator = dict()
        for y in range(1, 2*Ly, 2):
            operator[(0, y, 0)] = 'X'
        logicals.append(operator)

        # X operators along z edges in z direction
        operator = dict()
        for z in range(1, 2*Lz, 2):
            operator[(0, 0, z)] = 'X'
        logicals.append(operator)

        return logicals

    def get_logicals_z(self):
        """Get the 3 logical Z operators."""
        Lx, Ly, Lz = self.size
        logicals = []

        # Z operators on x edges forming surface normal to x (yz plane).
        operator = dict()
        for y in range(0, 2*Ly, 2):
            for z in range(0, 2*Lz, 2):
                operator[(1, y, z)] = 'Z'
        logicals.append(operator)

        # Z operators on y edges forming surface normal to y (zx plane).
        operator = dict()
        for z in range(0, 2*Lz, 2):
            for x in range(0, 2*Lx, 2):
                operator[(x, 1, z)] = 'Z'
        logicals.append(operator)

        # Z operators on z edges forming surface normal to z (xy plane).
        operator = dict()
        for x in range(0, 2*Lx, 2):
            for y in range(0, 2*Ly, 2):
                operator[(x, y, 1)] = 'Z'
        logicals.append(operator)

        return logicals

    def stabilizer_representation(self, location, rotated_picture=False):
        representation = super().stabilizer_representation(location, rotated_picture, json_file='toric3d.json')

        x, y, z = location
        if not rotated_picture and self.stabilizer_type(location) == 'face':
            if z % 2 == 0:  # xy plane
                representation['params']['normal'] = [0, 0, 1]
            elif x % 2 == 0:  # yz plane
                representation['params']['normal'] = [1, 0, 0]
            else:  # xz plane
                representation['params']['normal'] = [0, 1, 0]

        if rotated_picture and self.stabilizer_type(location) == 'face':
            if z % 2 == 0:
                representation['params']['normal'] = [0, 0, 1]
            elif x % 2 == 0:
                representation['params']['normal'] = [1, 0, 0]
            else:
                representation['params']['normal'] = [0, 1, 0]

        return representation
    
    def qubit_representation(self, location, rotated_picture=False):
        representation = super().qubit_representation(location, rotated_picture, json_file='toric3d.json')
        
        return representation

## Create GUI config file for the code

You might then want to add JSON configuration file for the GUI representation, which should look like the following:

In [3]:
json_dict = {
    "MyToric3DCode": {
        "qubits": {
            "kitaev": {
                "object": "cylinder",
                "color": {
                    "I": "pink",
                    "X": "red",
                    "Y": "green",
                    "Z": "blue"
                },
                "opacity": {
                    "activated": {
                        "min": 1,
                        "max": 1
                    },
                    "deactivated": {
                        "min": 0.1,
                        "max": 0.6
                    }
                },
                "params": {
                    "length": 2,
                    "radius": 0.1,
                    "angle": 0
                }
            },
            "rotated": {
                "object": "sphere",
                "color": {
                    "I": "white",
                    "X": "red",
                    "Y": "green",
                    "Z": "blue"
                },
                "opacity": {
                    "activated": {
                        "min": 1,
                        "max": 1
                    },
                    "deactivated": {
                        "min": 0.1,
                        "max": 0.4
                    }
                },
                "params": {
                    "radius": 0.2
                }
            }
        },
        "stabilizers": {
            "kitaev": {
                "vertex": {
                    "object": "sphere",
                    "color": {
                        "activated": "gold",
                        "deactivated": "white"
                    },
                    "opacity": {
                        "activated": {
                            "min": 1,
                            "max": 1
                        },
                        "deactivated": {
                            "min": 0.1,
                            "max": 0.6
                        }
                    },
                    "params": {"radius": 0.2}
                },
                "face": {
                    "object": "rectangle",
                    "color": {
                        "activated": "gold",
                        "deactivated": "blue"
                    },
                    "opacity": {
                        "activated": {
                            "min": 0.6,
                            "max": 0.6
                        },
                        "deactivated": {
                            "min": 0,
                            "max": 0
                        }
                    },
                    "params": {
                        "w": 1.5,
                        "h": 1.5,
                        "normal": [1, 0, 0],
                        "angle": 0
                    }
                }
            },
            "rotated": {
                "vertex": {
                    "object": "octahedron",
                    "color": {
                        "activated": "orange",
                        "deactivated": "orange"
                    },
                    "opacity": {
                        "activated": {
                            "min": 0.9,
                            "max": 0.9
                        },
                        "deactivated": {
                            "min": 0.1,
                            "max": 0.3
                        }
                    },
                    "params": {
                        "length": 1,
                        "angle": 0
                    }
                },
                "face": {
                    "object": "rectangle",
                    "color": {
                        "activated": "gold",
                        "deactivated": "gold"
                    },
                    "opacity": {
                        "activated": {
                            "min": 0.9,
                            "max": 0.9
                        },
                        "deactivated": {
                            "min": 0.1,
                            "max": 0.3
                        }
                    },
                    "params": {
                        "w": 1.4142,
                        "h": 1.4142,
                        "normal": [0,0,1],
                        "angle": 0.7854
                    }
                }
            }
        }
    }
}

In [4]:
with open('toric3d.json', 'w') as f:
    json.dump(json_dict, f)

## Start the GUI

You can then add your code to the GUI

In [5]:
gui = GUI()
gui.add_code(MyToric3DCode, 'My Toric 3D')
gui.run(port=5001)

 * Serving Flask app 'panqec.gui._gui' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2022 16:15:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:43] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "GET /2d HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "GET /js/shapes.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "GET /js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "GET /js/topologicalCode.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "POST /model-names HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:45] "POST /code-data HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:46] "POST /new-errors HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:47] "POST /decode HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:49] "POST /code-data HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2022 16:15:49] "POST /new-errors HTTP/1.1" 200 -
127.0.0.1 - - [0